# Speech Emotion Recognition - CS4120 Final project 
By Jesse E. and Alina G. 

In [2]:
# imports 
import os 
from tqdm import tqdm # to help with acknowledging runtimes...
import librosa # maybe need to import scipy (?)
import numpy as np
import utils
import speech_recognition as sr

In [3]:
path = './audio_data'
audio_folders = os.listdir(path)

audio_files = {} # save all filenames so we can use for text :))
audio_data = {} # save .wav data loaded in from librosa 

# get all filenames and load in wavs...
for folder in audio_folders: 
    curr_path = path + '/' + folder
    files = os.listdir(curr_path)
    audio_files[folder] = files

    data = utils.load_wavs(curr_path, files)
    audio_data[folder] = data

The above saves the `.wav` files in the following format: `{actor_folder: [((sig,rate), label), ...]}` in the audio_data dictionary. We can use `utils.seperate_tups()` to extract the sig,rate pairing and labels in each key/value pair. 
Below we will attempt to read in the audio and convert it to text. 

In [4]:
audio_text = {} # get audio text

for folder,files in tqdm(audio_files.items()): # takes approx 25 mins 
    curr_path = path + '/' + folder
    texts = utils.get_audio_text(curr_path, files)
    audio_text[folder] = texts

 56%|█████▌    | 14/25 [15:38<11:23, 62.14s/it]